# **학습에 필요한 파일 및 라이브러리 준비**

In [ ]:
import os
import yaml
from glob import glob
from google.colab import drive
import shutil

In [ ]:
# 드라이브 마운트
drive.mount('/content/drive')

# 데이터셋 압축파일 풀기
os.makedirs('/content/yolo_dataset/')
!unzip -qq '/content/drive/MyDrive/Colab Notebooks/파이널_ZBDS/dataset_main.zip' -d '/content/yolo_dataset/'

# yolov5 github clone
!git clone https://github.com/ultralytics/yolov5.git

# yolov5 구동에 필요한 필수 라이브러리 설치
%cd /content/yolov5/
!pip install -r requirements.txt

Mounted at /content/drive
Cloning into 'yolov5'...
remote: Enumerating objects: 15829, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 15829 (delta 21), reused 26 (delta 5), pack-reused 15769
Receiving objects: 100% (15829/15829), 14.59 MiB | 8.47 MiB/s, done.
Resolving deltas: 100% (10835/10835), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


In [ ]:
# 이미지 경로 설정
dataset_path = '/content/yolo_dataset/dataset_main/'
train_img_list = glob(dataset_path + '/train/images/*.jpg')
valid_img_list = glob(dataset_path + '/valid/images/*.jpg')
test_img_list = glob(dataset_path + '/test/images/*.jpg')

print('train_img_list : %5s | path : %50s' %(len(train_img_list), dataset_path + '/train/images/'))
print('valid_img_list : %5s | path : %50s' %(len(valid_img_list), dataset_path + '/valid/images/'))
print('test_img_list  : %5s | path : %50s' %(len(test_img_list), dataset_path + '/test/images/'))

train_img_list : 14494 | path :  /content/yolo_dataset/dataset_main//train/images/
valid_img_list :  1330 | path :  /content/yolo_dataset/dataset_main//valid/images/
test_img_list  :   695 | path :   /content/yolo_dataset/dataset_main//test/images/


In [ ]:
# yolov5s 학습에 필요한 yaml 데이터 생성
if not 'yolo_txt' in os.listdir(dataset_path):
    os.mkdir(dataset_path + '/yolo_txt')

txt_dir = '/content/yolo_dataset/dataset_main/yolo_txt/'

with open(txt_dir + 'train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open(txt_dir + 'valid.txt', 'w') as f:
  f.write('\n'.join(valid_img_list) + '\n')

with open(txt_dir + 'test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

yaml_data = {"names":['back','bottom','error','front','hand','head','side'], #
             "nc" : 7, # 클래스 수
             "path" : txt_dir, # root 경로
             "train" : os.path.join(txt_dir, "train.txt"), # train.txt 경로
             "val" : os.path.join(txt_dir, "valid.txt"), # valid.txt 경로
             "test" : os.path.join(txt_dir,"test.txt") # test.txt 경로
             }

with open(os.path.join(txt_dir, "data.yaml"), "w") as f:
  yaml.dump(yaml_data, f)

# **YOLOv5s 모델 학습**

In [ ]:
%cd /content/yolov5
shutil.copyfile('/content/drive/MyDrive/Colab Notebooks/파이널_ZBDS/yolov5s.pt', '/content/yolov5/yolov5s.pt')
!python train.py --img 640 \
                 --batch 48 \
                 --epochs 20 \
                 --data /content/yolo_dataset/dataset_main/yolo_txt/data.yaml \
                 --cfg ./models/yolov5s.yaml \
                 --weights ./yolov5s.pt \
                 --exist-ok

/content/yolov5
WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

train: weights=./yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/yolo_dataset/dataset_main/yolo_txt/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=48, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_

# **YOLOv5s 모델 테스트**

In [ ]:
!unzip -qq '/content/drive/MyDrive/Colab Notebooks/파이널_ZBDS/requirements.zip' -d '/content'

In [ ]:
!python val.py --data /content/yolo_dataset/dataset_main/yolo_txt/data.yaml\
               --weights /content/custom_yolov5s.onnx\
               --exist-ok

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

val: data=/content/yolo_dataset/dataset_main/yolo_txt/data.yaml, weights=['/content/custom_yolov5s.onnx'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=True, half=False, dnn=False
YOLOv5 🚀 v7.0-196-gacdf73b Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-S